#### Initial Segmentation and Mapping
- Segments images, enlarges them, and enhances their quality.
- Generates a **row mapping list** and **coordinates list** for all segmented images.
- Assigns a **row number** to each segmented image based on its position on the page for further processing.

In [58]:
import importlib as lib

import segmentation, save_and_load, mapping, identifications, modifications, generators, config

lib.reload(segmentation)
lib.reload(save_and_load)
lib.reload(mapping)
lib.reload(identifications)
lib.reload(modifications)
lib.reload(generators)
lib.reload(config)

<module 'config' from 'c:\\Users\\Admin\\Desktop\\Kernversion Backend\\config.py'>

In [4]:
from segmentation import extract_alphabets
from config import PATHS

pdf_path = "Analysis/multiple_page.pdf"
initial_segmentation_folder = PATHS['initial_segmentation']
all_coordinates, all_row_mappings = extract_alphabets(pdf_path, initial_segmentation_folder)

Page 0 Row Mapping: [(1, 71, 78), (2, 105, 116), (3, 121, 136), (4, 144, 147), (5, 154, 170), (6, 174, 177), (7, 182, 188), (8, 221, 223), (9, 255, 261), (10, 288, 320), (11, 325, 328), (12, 337, 349), (13, 356, 359), (14, 363, 369), (15, 377, 386), (16, 392, 405), (17, 416, 419), (18, 426, 441), (19, 443, 445), (20, 446, 448), (21, 449, 452), (22, 453, 459), (23, 501, 504), (24, 541, 553), (25, 581, 590), (26, 596, 598), (27, 621, 629), (28, 635, 649), (29, 655, 658), (30, 665, 680), (31, 685, 689), (32, 693, 699), (33, 706, 715), (34, 721, 737), (35, 742, 745), (36, 752, 767), (37, 774, 776), (38, 779, 786)]
Page 0 Coordinates: [(472, 783, 7, 6), (318, 782, 7, 7), (166, 782, 6, 7), (394, 781, 6, 7), (243, 780, 11, 10), (90, 779, 6, 12), (493, 774, 17, 5), (504, 759, 7, 11), (431, 759, 8, 12), (391, 759, 15, 13), (354, 759, 14, 12), (319, 759, 12, 16), (240, 758, 13, 12), (127, 758, 17, 13), (88, 758, 8, 12), (164, 757, 18, 20), (468, 754, 16, 19), (281, 754, 14, 18), (492, 753, 9, 15

#### User Input: Select Range of Row Numbers
- Accepts user input to specify the range of row numbers containing the **full composition**.
- Processes only the selected rows for further analysis and conversion.

In [5]:
first_row = 24
last_row = 72

In [6]:
from generators import copy_images_in_row_range

working_composition_folder = PATHS['working_composition']

copy_images_in_row_range(initial_segmentation_folder, working_composition_folder, first_row, last_row)

#### Select the metadata (Raag, Taal, Lay etc.)

In [7]:
raag_name = "Yaman"
taal_name = "Ektaal"
lay = "Vilambit"
source_name = "hindustani-sangeet-paddhati-kramik-pustak-malika-part-2"

#### Show valid positions for the sam beat and take the input from user for sam beat

In [8]:
from identifications import calculate_valid_sam_positions

valid_positions = calculate_valid_sam_positions(taal_name)
print(f"Valid Sam positions for {taal_name}: {valid_positions}")

Valid Sam positions for Ektaal: [1, 3, 5, 7, 9, 11]


In [9]:
sthayee_sam = 5
antara_sam = 5

In [10]:
from save_and_load import save_composition_metadata

save_composition_metadata(raag_name, taal_name, lay, sthayee_sam, antara_sam, source_name=source_name)

In [11]:
from save_and_load import get_metadata_field

taal = get_metadata_field(field="taal_name")
sthayee_sam_beat = get_metadata_field(field="sthayee.sam_beat")

print(f"Taal name: {taal}")
print(f"First sam beat in Sthayee: {sthayee_sam_beat}")

Taal name: Ektaal
First sam beat in Sthayee: 5


In [12]:
from save_and_load import get_taal_data, get_taal_field

taal_data = get_taal_data(taal)
beat = get_taal_field(taal_name=taal, field_name="beat_count")

print(f"Beat Count: {beat}")
print(taal_data)

Beat Count: 12
{'beat_count': 12, 'divisions': [2, 2, 2, 2, 2, 2], 'vibhaag': ['X', '0', '2', '0', '3', '4'], 'time_signature': '12/8'}


In [13]:
from identifications import calculate_divisions_and_vibhaag

# Calculate divisions and vibhaag
new_divisions, new_vibhaag = calculate_divisions_and_vibhaag(taal, sthayee_sam_beat)

# Define the beat count (size of the lists) from taal information
beat_count = get_taal_field(taal_name=taal, field_name="beat_count")

# Print the results
print(f"\nTaal: {taal}")
print(f"Total number of beats: {beat_count}")
print(f"Sam (X) at beat: {sthayee_sam_beat}")
print(f"New Divisions: {new_divisions}")
print(f"New Vibhaag: {new_vibhaag}")


Taal: Ektaal
Total number of beats: 12
Sam (X) at beat: 5
New Divisions: [2, 2, 2, 2, 2, 2]
New Vibhaag: ['3', '4', 'X', '0', '2', '0']


#### Calculate Image Count for Each Row
- Counts the number of images in each row by processing the segmented image files.

In [14]:
from mapping import get_row_image_counts

# Get image counts for all rows
row_image_count = get_row_image_counts(working_composition_folder)

#### Identify Rows with Sam, Taalis, and Khali
- Creates a list of rows that could represent Sam, Taalis, and Khali based on the number of images in each row.
- Uses the given taal information to determine the expected number of images for these rows.

In [15]:
from identifications import get_sam_and_taalis_rows

sam_and_taalis_rows = get_sam_and_taalis_rows(row_image_count, taal_name=taal)
print(f"Rows with sam and taalis: ",sam_and_taalis_rows)

Rows with sam and taalis:  [32, 38, 44, 52, 58, 65, 71]


#### User Correction: Update Identified Rows
- Allows the user to review and correct the automatically identified rows for Sam, Taalis, and Khali.
- Users can:
  - Add missing rows.
  - Remove incorrectly identified rows.
  - Update the list to ensure accuracy.
- Uses the corrected row numbers to create **subgroups**, where each subgroup includes:
  - A **swar row**.
  - Optional **kann swar**, **lyrics**, and **articulation rows**.

In [16]:
sam_and_taalis_rows = [32, 38, 44, 52, 58, 65, 71] # multiple pages full composition - yaman-ektaal-vilambit

#### Initial Subgroup Creation and Assigning Column Numbers
- Finds the first valid row in each subgroup.
- Assigns `extra` for images that are not part of the composition (noise and metadata).
- Assigns `column numbers` for valid images that are part of the composition.

In [17]:
from mapping import assign_column_numbers

subgroup_ranges = assign_column_numbers(all_row_mappings, first_row, sam_and_taalis_rows)
print("Subgroup Ranges:", subgroup_ranges)

Subgroup range: [24, 32]
First valid row in subgroup:  26
Subgroup range: [33, 38]
First valid row in subgroup:  33
Subgroup range: [39, 44]
First valid row in subgroup:  39
Subgroup range: [45, 52]
First valid row in subgroup:  47
Subgroup range: [53, 58]
First valid row in subgroup:  53
Subgroup range: [59, 65]
First valid row in subgroup:  59
Subgroup range: [66, 71]
First valid row in subgroup:  66
Subgroup Ranges: [(26, 31), (33, 37), (39, 43), (47, 51), (53, 57), (59, 64), (66, 70)]


#### Update the subgroup if required
- When there is subgroup not having corresponding sam and taalis row (1st subgroup in most of the case)

In [18]:
from modifications import update_subgroups

updated_subgroups = update_subgroups(subgroup_ranges)

for start, end in updated_subgroups:
    print(f"Updated Subgroup range: [{start}, {end}]")

Updated Subgroup range: [26, 31]
Updated Subgroup range: [33, 37]
Updated Subgroup range: [39, 43]
Updated Subgroup range: [47, 51]
Updated Subgroup range: [53, 57]
Updated Subgroup range: [59, 64]
Updated Subgroup range: [66, 70]


In [19]:
# Assign back the updated subgroups
subgroup_ranges = updated_subgroups

for start, end in subgroup_ranges:
    print(f"Updated Subgroup range: [{start}, {end}]")

Updated Subgroup range: [26, 31]
Updated Subgroup range: [33, 37]
Updated Subgroup range: [39, 43]
Updated Subgroup range: [47, 51]
Updated Subgroup range: [53, 57]
Updated Subgroup range: [59, 64]
Updated Subgroup range: [66, 70]


#### Section Identification
- Identifies row numbers belonging to specific sections:
  - **Kann Swar**
  - **Swar**
  - **Lyrics**
  - **Articulation**
- Organizes rows into their respective sections for further processing.

In [20]:
from identifications import classify_rows_in_subgroups
from save_and_load import save_row_categories

row_categories = classify_rows_in_subgroups(subgroup_ranges)

# save the row numbers category wise
save_row_categories(row_categories)

Articulation Rows:  [29, 35, 37, 49, 55, 57, 61, 64, 68, 70]
Kann Swar Rows:  [33, 53, 59, 66]
Swar Rows:  [34, 54, 60, 67]
Lyrics Rows:  [36, 56, 62, 69]


#### User Correction and Selection
- Allows user to **correct** identified sections (**Kann Swar**, **Swar**, **Lyrics**, **Articulation**).
- Enables selection of **Sthayee** and **Antara** rows.
- Ensures accurate grouping and processing of rows based on user input.

In [ ]:
row_categories = {
    "articulation": [29, 31, 35, 37, 49, 51, 55, 57, 61, 64, 68, 70],
    "kann_swar": [27, 33, 40, 47, 53, 59, 66],
    "swar": [28, 34, 41, 48, 54, 60, 67],
    "lyrics": [30, 36, 43, 50, 56, 62, 69],
    "sthayee": 25,
    "antara": 46
}

In [22]:
save_row_categories(row_categories)

In [23]:
from save_and_load import load_row_categories

row_categories = load_row_categories()

# Extract specific row categories
articulation_rows = row_categories.get("articulation", [])
kann_swar_rows = row_categories.get("kann_swar", [])
swar_rows = row_categories.get("swar", [])
lyrics_rows = row_categories.get("lyrics", [])
sthyaee = row_categories.get("sthayee")
antara = row_categories.get("antara")

# Print the results
print("Articulation Rows: ", articulation_rows)
print("Kann Swar Rows: ", kann_swar_rows)
print("Swar Rows: ", swar_rows)
print("Lyrics Rows: ", lyrics_rows)
print("Sthayee ", sthyaee)
print("Antara: ", antara)

Articulation Rows:  [29, 31, 35, 37, 49, 51, 55, 57, 61, 64, 68, 70]
Kann Swar Rows:  [27, 33, 40, 47, 53, 59, 66]
Swar Rows:  [28, 34, 41, 48, 54, 60, 67]
Lyrics Rows:  [30, 36, 43, 50, 56, 62, 69]
Sthayee  25
Antara:  46


In [24]:
# Print the taal and beat count to ensure 
print(f"\nTaal: {taal}")
print(f"Total number of beats: {beat_count}")


Taal: Ektaal
Total number of beats: 12


#### Initial List Creation and Mapping
- Creates lists for each subgroup:
  - **Kann Swar**
  - **Swar**
  - **Swar Articulation Checks**
  - **Lyrics**
  - **Lyrics Articulation Checks**
- Stores image file paths in the respective lists.

#### Kann Swar and Swar Mapping
- **Case 1: Explicit Kann Swar Row**  
  - Maps **Kann Swar** to its corresponding **Swar** directly.
- **Case 2: Hidden Kann Swar in Swar Row**  
  - Identifies **Kann Swar** images present in the **Swar** row and moves them to the **Kann Swar** list.
  - Detects and segments composite images containing both **Kann Swar** and **Swar**.
  - Stores the segmented **Kann Swar** image path in the **Kann Swar** list.
  - Stores the segmented **Swar** image path in the **Swar** list.

#### Articulation Mapping
- Maps **Swar Articulation** rows to their corresponding **Swar**.
- Maps **Lyrics Articulation** rows to their corresponding **Lyrics**.

In [35]:
from generators import generate_lists_in_subgroups
import os
from filename_utils import get_image_details_with_path
from collections import defaultdict

# Load all image filenames and extract their information
image_files = os.listdir(working_composition_folder)
image_info = [get_image_details_with_path(f, working_composition_folder) for f in image_files]
image_info = [info for info in image_info if info is not None]

# Organize images by row and column
row_col_images = defaultdict(lambda: defaultdict(list))
for info in image_info:
    page_num, row_num, col_num, x, y, width, height, image_path = info
    row_col_images[row_num][col_num].append((x, y, width, height, image_path))

# Function to generate list of lists for all sections in each subgroup
subgroups = generate_lists_in_subgroups(subgroup_ranges, row_col_images, beat_count)

In [36]:
from save_and_load import save_lists_in_subgroups

save_lists_in_subgroups(subgroups)

In [37]:
from save_and_load import load_lists_in_subgroups

subgroups = load_lists_in_subgroups()

# Print the results for each subgroup
for subgroup_range, results in subgroups.items():
    print(f"Subgroup Range: {subgroup_range}")
    print(f"Kann Swar List: {results['kann_swar_list']}")
    print(f"Swar List: {results['swar_list']}")
    print(f"Swar Articulation Checks: {results['swar_articulation_checks']}")
    print(f"Lyrics List: {results['lyrics_list']}")
    print(f"Lyrics Articulation Checks: {results['lyrics_articulation_checks']}")
    print("-" * 80)

Subgroup Range: (26, 31)
Kann Swar List: [['Analysis\\multiple_page\\working_composition\\0_row27_col1_x80_y624_w8_h9.png', 'Analysis\\multiple_page\\working_composition\\0_row27_col1_x94_y624_w9_h9.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col2_x112_y621_w10_h11.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col3_x159_y624_w7_h9.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col4_x197_y624_w7_h9.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col5_x236_y625_w10_h9.png'], [], [], [], [], [], ['Analysis\\multiple_page\\working_composition\\0_row27_col11_x465_y621_w10_h13.png'], []]
Swar List: [['Analysis\\multiple_page\\working_composition\\0_row28_col1_x82_y637_w28_h15.png'], ['Analysis\\multiple_page\\working_composition\\0_row28_col2_x117_y638_w13_h14.png', 'Analysis\\multiple_page\\working_composition\\0_row28_col2_x120_y638_w25_h23.png'], ['Analysis\\multiple_page\\working_composition\\0_row28_col3_x161_y642_w11_h1

#### Meend Segmentation and List Creation
- Segments composite **Meend** and **Kann Swar** images.
- Stores the **Kann Swar** image in its correct position in the **Kann Swar List** based on coordinates.
- Creates a separate **Meend List** to store:
  - **Start** and **end positions** of each **Meend** corresponding to the **Swar** row.
- Detects **Meend** in the **Kann Swar** row.
- If **Meend** is present:
    - Update the **Meend List**
    - Remove the image path from **Kann Swar List**

In [38]:
from generators import update_kann_swar_and_generate_meend_lists

update_kann_swar_and_generate_meend_lists()

In [39]:
subgroups = load_lists_in_subgroups()

# Print the updated results for each subgroup
for subgroup_range, results in subgroups.items():
    print(f"Subgroup Range: {subgroup_range}")
    print(f"Kann Swar List: {results['kann_swar_list']}")
    print(f"Meend List: {results['meend_list']}")
    print(f"Swar List: {results['swar_list']}")
    print(f"Lyrics List: {results['lyrics_list']}")
    print(f"Swar Articulation Checks: {results['swar_articulation_checks']}")
    print(f"Lyrics Articulation Checks: {results['lyrics_articulation_checks']}")
    print("-" * 80)

Subgroup Range: (26, 31)
Kann Swar List: [['Analysis\\multiple_page\\working_composition\\0_row27_col1_x80_y624_w8_h9.png', 'Analysis\\multiple_page\\working_composition\\0_row27_col1_x94_y624_w9_h9.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col2_x112_y621_w10_h11.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col3_x159_y624_w7_h9.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col4_x197_y624_w7_h9.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col5_x236_y625_w10_h9.png'], [], [], [], [], [], ['Analysis\\multiple_page\\working_composition\\0_row27_col11_x465_y621_w10_h13.png'], []]
Meend List: ['', '', '', '', '', '', '', '', '', '', '', '']
Swar List: [['Analysis\\multiple_page\\working_composition\\0_row28_col1_x82_y637_w28_h15.png'], ['Analysis\\multiple_page\\working_composition\\0_row28_col2_x117_y638_w13_h14.png', 'Analysis\\multiple_page\\working_composition\\0_row28_col2_x120_y638_w25_h23.png'], ['Analysis\\mult

#### Final Segmentation and List Finalization
- Performs further segmentation to finalize lists for each subgroup.

#### Articulation Detection and Segmentation
- Detects and segments **articulation** from:
  - **Swar Row**: Updates the **Swar Articulation Checks** list.
  - **Lyrics Row**: Updates the **Lyrics Articulation Checks** list.

#### Word Segmentation
- Applies **word segmentation** to:
  - **Swar Row**: For images corresponding to **Swar Articulation Checks**.
  - **Lyrics Row**: For images corresponding to **Lyrics Articulation Checks**.

In [40]:
from modifications import finalize_segmentation_and_lists

finalize_segmentation_and_lists()

In [41]:
subgroups = load_lists_in_subgroups()

# Print the updated results for each subgroup
for subgroup_range, results in subgroups.items():
    print(f"Subgroup Range: {subgroup_range}")
    print(f"Kann Swar List: {results['kann_swar_list']}")
    print(f"Meend List: {results['meend_list']}")
    print(f"Swar List: {results['swar_list']}")
    print(f"Lyrics List: {results['lyrics_list']}")
    print(f"Swar Articulation Checks: {results['swar_articulation_checks']}")
    print(f"Lyrics Articulation Checks: {results['lyrics_articulation_checks']}")
    print("-" * 80)

Subgroup Range: (26, 31)
Kann Swar List: [['Analysis\\multiple_page\\working_composition\\0_row27_col1_x80_y624_w8_h9.png', 'Analysis\\multiple_page\\working_composition\\0_row27_col1_x94_y624_w9_h9.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col2_x112_y621_w10_h11.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col3_x159_y624_w7_h9.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col4_x197_y624_w7_h9.png'], ['Analysis\\multiple_page\\working_composition\\0_row27_col5_x236_y625_w10_h9.png'], [], [], [], [], [], ['Analysis\\multiple_page\\working_composition\\0_row27_col11_x465_y621_w10_h13.png'], []]
Meend List: ['', '', '', '', '', '', '', '', '', '', '', '']
Swar List: [['Analysis\\multiple_page\\working_composition_segmented\\0_row28_col1_x82_y637_w28_h15_seg1.png', 'Analysis\\multiple_page\\working_composition_segmented\\0_row28_col1_x82_y637_w28_h15_seg2.png'], ['Analysis\\multiple_page\\working_composition\\0_row28_col2_x117_y638_w1

#### Prediction for Kann Swar and Swar Rows
- Predicts results for:
  - **Kann Swar Row Images**: Uses the **Kann Swar List** for predictions.
  - **Swar Row Images**: Uses the **Swar List** for predictions.
- Generates predictions for each subgroup based on the respective lists.

In [46]:
from save_and_load import load_classes

classes = load_classes()

print(classes[0])

saa


In [52]:
from generators import generate_predictions

subgroups = load_lists_in_subgroups()
predicted_results = generate_predictions(subgroups)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step


In [ ]:
from save_and_load import save_predictions

save_predictions(predicted_results)

In [67]:
from save_and_load import load_predictions

def print_predictions(predictions):
    """Prints results with original formatting"""
    for subgroup_range, results in predictions.items():
        print(f"\nSubgroup Range: {subgroup_range}")
        print(f"Predicted Kann Swar List: {results['kann_swar']}")
        print(f"Predicted Swar List: {results['swar']}") 
        print("-" * 80)

predictions = load_predictions()

if predictions:
    print_predictions(predictions)


Subgroup Range: (26, 31)
Predicted Kann Swar List: [['pa', 'dha'], ['ma#'], ['pa'], ['saa'], ['ga'], [], [], [], [], [], ['ma#'], []]
Predicted Swar List: [["saa'", 'ma#,'], ['ga', 'pa,'], ['ga'], ['ga', 'pa'], ['re'], ['ga'], ['ga'], ['ga'], ['ga'], ['re'], ['ga'], ['ma#']]
--------------------------------------------------------------------------------

Subgroup Range: (33, 37)
Predicted Kann Swar List: [[], ['ga'], [], [], ['ni,'], [], [], [], [], [], [], [], []]
Predicted Swar List: [['pa'], ['re'], ['ni,', 're'], ['saa'], ['saa'], ['re'], ['ga'], ['re'], ['saa'], ['pa,'], ['saa'], ['ni,'], ['ni,', 'dha,']]
--------------------------------------------------------------------------------

Subgroup Range: (39, 43)
Predicted Kann Swar List: [['dha'], [], [], [], [], [], ['pa'], [], ['ga'], [], [], []]
Predicted Swar List: [['ni'], ['dha,'], ['pa'], ['-'], ['pa'], ['-'], ['ga'], ['pa'], ['re'], ['ni,'], ['re'], ['saa']]
-----------------------------------------------------------------

In [64]:
from identifications import calculate_divisions_and_vibhaag
from save_and_load import get_metadata_field, get_taal_field

# taal = get_metadata_field(field='taal_name')
# sthayee_sam = get_metadata_field(field='sthayee.sam_beat')

taal = "Ektaal"
sthayee_sam_beat = 9

# Calculate divisions and vibhaag
new_divisions, new_vibhaag = calculate_divisions_and_vibhaag(taal, sthayee_sam_beat)

# Define the beat count (size of the lists) from taal information
beat_count = get_taal_field(taal, field_name="beat_count")

# Print the results
print(f"\nTaal: {taal}")
print(f"Total number of beats: {beat_count}")
print(f"Sam (X) at beat: {sthayee_sam_beat}")
print(f"New Divisions: {new_divisions}")
print(f"New Vibhaag: {new_vibhaag}")


Taal: Ektaal
Total number of beats: 12
Sam (X) at beat: 9
New Divisions: [2, 2, 2, 2, 2, 2]
New Vibhaag: ['2', '0', '3', '4', 'X', '0']


#### Under construction!

In [71]:
# Function to generate kern code based on user input
def generate_kern_code(raag, taal, lay):
    
    time_signature = get_taal_field(taal_name=taal, field_name='time_signature')
    
    # Set tempo based on lay
    # Tempo mapping
    tempo_map = {
        "vilambit": 60,
        "madhya": 90,
        "drut": 150
    }

    tempo = tempo_map.get(lay.lower(), 60)  # Default to vilambit

    # Generate the kern code
    return (
        f"!!!raag: {raag}\n"
        f"!!!taal: {taal}\n"
        f"!!!lay: {lay}\n"
        f"**kern\n"
        f"*M{time_signature}\n"
        f"*MM{tempo}\n"
        f"*Isitar\n"
        f"*c:\n"
        f"!! Sthayee\n"
    )

In [72]:
raag = get_metadata_field(field='raag_name')
lay = get_metadata_field(field='lay')

kern_code = generate_kern_code(raag, taal, lay)

print(kern_code)

!!!raag: Yaman
!!!taal: Ektaal
!!!lay: Vilambit
**kern
*M12/8
*MM60
*Isitar
*c:
!! Sthayee



In [2]:
from save_and_load import load_predictions

def flatten_predictions(predictions):
    """Flattens all predictions into single lists of swar and kann swar"""
    flat_swar = []
    flat_kann = []
    
    for subgroup_data in predictions.values():
        flat_swar.extend(subgroup_data["swar"])
        flat_kann.extend(subgroup_data["kann_swar"])
    
    return flat_swar, flat_kann

def print_flattened_predictions(flat_swar, flat_kann):
    """Prints the flattened lists"""
    # print("\nFlattened Swar List:")
    # for i, swar in enumerate(flat_swar):
    #     print(f"{i}: {swar}")
    
    # print("\nFlattened Kann Swar List:")
    # for i, kann in enumerate(flat_kann):
    #     print(f"{i}: {kann}")

    print(flat_kann)
    print(flat_swar)

# Load and process predictions
predictions = load_predictions()

if predictions:
    flat_swar, flat_kann = flatten_predictions(predictions)
    print_flattened_predictions(flat_swar, flat_kann)

[['pa', 'dha'], ['ma#'], ['pa'], ['saa'], ['ga'], [], [], [], [], [], ['ma#'], [], [], ['ga'], [], [], ['ni,'], [], [], [], [], [], [], [], [], ['dha'], [], [], [], [], [], ['pa'], [], ['ga'], [], [], [], ['ma#'], [], ["ma#'"], [], [], [], [], [], [], [], ['ni'], ['ni'], ['ni'], [], [], [], [], [], [], [], [], ["saa'"], [], [], ['ma#'], [], [], ['ni'], [], [], [], [], [], ["ma#'"], [], [], ['dha'], [], [], [], [], [], [], [], [], [], [], ['re'], [], []]
[["saa'", 'ma#,'], ['ga', 'pa,'], ['ga'], ['ga', 'pa'], ['re'], ['ga'], ['ga'], ['ga'], ['ga'], ['re'], ['ga'], ['ma#'], ['pa'], ['re'], ['ni,', 're'], ['saa'], ['saa'], ['re'], ['ga'], ['re'], ['saa'], ['pa,'], ['saa'], ['ni,'], ['ni,', 'dha,'], ['ni'], ['dha,'], ['pa'], ['-'], ['pa'], ['-'], ['ga'], ['pa'], ['re'], ['ni,'], ['re'], ['saa'], ['pa'], ['ga'], ['pa'], ['dha', 'pa'], ["saa'"], [','], ["saa'"], ["saa'"], ['-'], ["saa'"], ["saa'"], ["saa'"], ["re'"], ["re'"], ["saa'"], ['ni'], ['pa'], ['ni'], ['-'], ['dha'], ['pa'], ['pa'], 

In [ ]:
def convert_to_kern(flat_kann, flat_swar, divisions, beat_count):
    swar_to_kern = {
        "saa": "c", "re": "d", "ga": "e", "ma": "f", "pa": "g", "dha": "a", "ni": "b",
        "re-": "d-", "ga-": "e-", "dha-": "a-", "ni-": "b-", "ma#": "f#",
        "saa'": "cc", "re'": "dd", "ga'": "ee", "ma'": "ff", "pa'": "gg", "dha'": "aa", "ni'": "bb",
        "re-'": "dd-", "ga-'": "ee-", "ma#'": "ff#",
        "ma,": "F", "ma#,": "F#", "pa,": "G", "dha-,": "A-", "dha,": "A", "ni-,": "B-", "ni,": "B",
        ",": ",", "-": "-"
    }
    
    kern_output = []
    total_beats = len(flat_kann)
    cumulative_divisions = []
    current_sum = 0
    
    while current_sum <= total_beats:
        for d in divisions:
            current_sum += d
            if current_sum >= total_beats:
                break
            cumulative_divisions.append(current_sum)

    print(f"cumulative_divisions: {cumulative_divisions}")
    first_empty_flag = True
    
    for i in range(total_beats):
        # Handle major divisions (beat_count)
        if i % beat_count == 0:
            kern_output.append(f"\n=={i // beat_count + 1}\n")
        
        # Handle subdivisions
        if i in cumulative_divisions and i % beat_count != 0:
            kern_output.append("\n=\n")
        
        # Process current beat
        kann = flat_kann[i]
        swar = flat_swar[i]
        
        # Handle empty lists
        if not kann and not swar and first_empty_flag:
            kern_output.append("4ryy\n")
            continue

        # Do not need to add rests at the last
        if not kann and not swar and not first_empty_flag:
            # Join and clean up newlines
            kern_str = ''.join(kern_output)
            kern_str = kern_str.replace('\n\n', '\n')  # Remove duplicate newlines
            return kern_str.strip(), kern_output

        # set flag to false to handle empty lists at the end of composition
        first_empty_flag = False
        
        # Process non-empty lists
        for idx in range(max(len(kann), len(swar))):
            # Process kann if exists
            if idx < len(kann):
                k = kann[idx]
                if k != '':
                    kern_output.append(f"{swar_to_kern[k]}q\n")
            
            # Process swar if exists
            if idx < len(swar):
                s = swar[idx]
                if s == '-':
                    handle_extension(kern_output)
                else:
                    # Handle parenthesized elements in swar
                    is_parenthesized = s.startswith('(') and s.endswith(')')
                    s_clean = s[1:-1] if is_parenthesized else s
                    duration = 4 * len(swar)
                    kern_code = f"{duration}{swar_to_kern[s_clean]}"
                    if is_parenthesized:
                        kern_code += 'S'
                    kern_output.append(f"{kern_code}\n")
    
    # Join and clean up newlines
    kern_str = ''.join(kern_output)
    kern_str = kern_str.replace('\n\n', '\n')  # Remove duplicate newlines
    return kern_str.strip(), kern_output

def handle_extension(kern_output):
    """Handle extension cases for '-' in swar"""
    if not kern_output:
        return
    
    # Pop division markers if they exist
    popped = []
    while kern_output and (kern_output[-1].strip().startswith('==') or kern_output[-1].strip() == '='):
        popped.append(kern_output.pop())
    
    if not kern_output:
        kern_output.extend(popped)
        return
    
    last = kern_output[-1].strip()
    
    # Check if the last element has 'S' suffix
    has_S_suffix = last.endswith('S')
    if has_S_suffix:
        last = last[:-1]  # Remove 'S' temporarily
    
    # Case 3: Ends with ']'
    if last.endswith(']'):
        kern_code = last[:-1]
        kern_output[-1] = kern_code
        modified = modify_kern_code(kern_code)
        if isinstance(modified, tuple):
            kern_output[-1] = modified[0] + ('S' if has_S_suffix else '') + '\n'
            kern_output.append(modified[1] + ('S' if has_S_suffix else '') + '\n')
        else:
            kern_output[-1] = modified + ('S' if has_S_suffix else '') + ']\n'
    
    # Case 4,5,6: Regular kern codes
    else:
        modified = modify_kern_code(last)
        if isinstance(modified, tuple):
            kern_output[-1] = modified[0] + ('S' if has_S_suffix else '') + '\n'
            kern_output.append(modified[1] + ('S' if has_S_suffix else '') + '\n')
        else:
            kern_output[-1] = modified + ('S' if has_S_suffix else '') + '\n'
    
    # Push back any popped division markers
    kern_output.extend(reversed(popped))

def modify_kern_code(kern_code):
    """Modify kern code according to extension rules"""
    # Split into number and code parts
    num_part = ''
    code_part = kern_code
    for i, c in enumerate(kern_code):
        if c.isdigit():
            num_part += c
        else:
            code_part = kern_code[i:]
            break
    
    if not num_part:
        return '2' + code_part
    
    num = int(num_part)
    
    if num == 4:
        return '2' + code_part
    elif num < 4:
        return f'{num}.' + code_part
    else:  # num > 4
        return (f'[{num}{code_part}', f'4{code_part}]')


# Example usage:
divisions = [4, 5, 2, 3]
beat_count = 14
flat_kann = [[], [], [], [], [], [], [], [], [], [], [], [], ['pa', '', 'dha'], [], [], [], ["saa'"], ['dha'], [], [], [], [], ['dha'], [], [], ["saa'"], [], [], ["saa'"], [], [], [], ['pa'], ['pa'], [], ['ma'], [], [], [], [], [], [], [], [], [], [], ['ni,'], [], [], ["saa'"], [], [], [], [], [], []]
flat_swar = [[], [], [], [], [], [], [], [], [], [], [], [], ['(ni)', 'ni', '(ni)'], ['-'], ['-'], ['ga'], ['ni'], ['ni'], ["saa'"], ['-'], ['-'], ['ni'], ['ni'], ["-"], ['-'], ["re'"], ["saa'"], ['-'], ['dha'], ['dha', 'ni-'], ['pa'], ['-'], ['ga'], ['ga'], ['ma', 're'], ['ga'], ['ma'], ['pa'], ['ma', 'ga'], ['ga'], ['(re)'], ['-'], ['saa'], ['-'], ['-'], ['saa'], ["saa'"], ["saa'"], ['-'], ['dha'], ['-'], ['ni-'], ['pa'], [], [], []]

kern_output, kp = convert_to_kern(flat_kann, flat_swar, divisions, beat_count)
print(kern_output)

cumulative_divisions: [4, 9, 11, 14, 18, 23, 25, 28, 32, 37, 39, 42, 46, 51, 53]
==1
4ryy
4ryy
4ryy
4ryy
=
4ryy
4ryy
4ryy
4ryy
4ryy
=
4ryy
4ryy
=
4ryy
gq
12bS
12b
aq
[12bS
2bS]
==2
4e
ccq
4b
aq
4b
=
2.cc
4b
aq
2.b
=
=
ccq
4dd
2cc
==3
ccq
4a
8a
8b-
2g
=
gq
4e
gq
4e
8f
8d
fq
4e
4f
=
4g
8f
8e
=
4e
2dS
==4
2.c
4c
=
Bq
4cc
2cc
ccq
2a
=
4b-
4g
=


In [74]:
print(kp)

['\n==1\n', '4ryy\n', '4ryy\n', '4ryy\n', '4ryy\n', '\n=\n', '4ryy\n', '4ryy\n', '4ryy\n', '4ryy\n', '4ryy\n', '\n=\n', '4ryy\n', '4ryy\n', '\n=\n', '4ryy\n', 'gq\n', '12bS\n', '12b\n', 'aq\n', '[12bS\n', '2bS]\n', '\n==2\n', '4e\n', 'ccq\n', '4b\n', 'aq\n', '4b\n', '\n=\n', '2.cc\n', '4b\n', 'aq\n', '2.b\n', '\n=\n', '\n=\n', 'ccq\n', '4dd\n', '2cc\n', '\n==3\n', 'ccq\n', '4a\n', '8a\n', '8b-\n', '2g\n', '\n=\n', 'gq\n', '4e\n', 'gq\n', '4e\n', '8f\n', '8d\n', 'fq\n', '4e\n', '4f\n', '\n=\n', '4g\n', '8f\n', '8e\n', '\n=\n', '4e\n', '2dS\n', '\n==4\n', '2.c\n', '4c\n', '\n=\n', 'Bq\n', '4cc\n', '2cc\n', 'ccq\n', '2a\n', '\n=\n', '4b-\n', '4g\n', '\n=\n']


In [78]:
def generate_sthayee_to_antara(kern_output, sthayee_vibhaag, antara_vibhaag):
    # count number of divisions passed in last beat cycle, next taali will be at this count index
    count_division = 0

    for i in reversed(range(len(kern_output))):
        if kern_output[i].startswith('\n=='):
            break
        elif kern_output[i] == '\n=\n':
            count_division += 1

    # get index of sam in vibhaag
    sthayee_sam_index = -1
    for i in range(len(sthayee_vibhaag)):
        if sthayee_vibhaag[i] == 'X':
            sthayee_sam_index = i
            break

    # get index of sam in antara
    antara_sam_index = -1
    for i in range(len(antara_vibhaag)):
        if antara_vibhaag[i] == 'X':
            antara_sam_index = i
            break
    
    i = 0
    while ((sthayee_sam_index + i) % len(sthayee_vibhaag)) != count_division:
        i += 1

    # find next taali index in antara
    next_taali_index = ((antara_sam_index + i) % len(sthayee_vibhaag))

    # starting from index as per count_division, number of divisions to repeat
    count_repeat_division = len(sthayee_vibhaag) - next_taali_index

    count = -1
    i = 0
    while count < count_division:
        item = kern_output[i]
        if item.startswith('\n==') or item == ('\n=\n'):
            count += 1
        i += 1

    
    # transition output starts from index i
    sthayee_to_antara_kern = []

    count = 0
    while count < count_repeat_division:
        item = kern_output[i]
        if item.startswith('\n==') or item == ('\n=\n'):
            count += 1
            item = '\n=\n'

        sthayee_to_antara_kern.append(item)
        i += 1

    # return sthayee_to_antara_kern

    # Join and clean up newlines
    kern_str = ''.join(sthayee_to_antara_kern)
    kern_str = kern_str.replace('\n\n', '\n')  # Remove duplicate newlines
    return kern_str.strip(), sthayee_to_antara_kern


# sthayee_vibhaag = ["3", "X", "2", "0"]
# antara_vibhaag = ["X", "2", "0", "3"]

sthayee_vibhaag = ['3', '4', 'X', '0', '2', '0']
antara_vibhaag = ['0', '3', '4', 'X', '0', '2']

sthayee_to_antara_kern, sthayee_antara_kern_output = generate_sthayee_to_antara(kp, sthayee_vibhaag, antara_vibhaag)

print(sthayee_to_antara_kern)

4ryy
gq
12bS
12b
aq
[12bS
2bS]
=
4e
ccq
4b
aq
4b
=
